In [1]:
import pandas as pd
import numpy as np
from langchain import PromptTemplate    
from langchain.prompts import ChatPromptTemplate
from langchain.chat_models import ChatOpenAI
import openai
import datetime
import random


import os
os.environ["OPENAI_API_KEY"] = "XXXXXX"
openai.api_key = os.getenv("OPENAI_API_KEY")
OPENAI_MODEL = "gpt-3.5-turbo-16k-0613"

# Initializing LLM Model

In [2]:
chat_model = ChatOpenAI(model_name = OPENAI_MODEL, temperature=0.7)

# Defining Prompt Templates

# Importing Script and Intention Calls

In [3]:
df_script = pd.read_excel('../../data/scripts_call_center_ee.xlsx', sheet_name='general_script1')
df_script

,Step,Agent Script,Customer Response Options
0,Greeting,"""Good [morning/afternoon/evening], thank you f...",-
1,Identify Issue,"""Could you please tell me about the issue you'...","""[Describe issue]"""
2,Confirm Account Details,"""May I have your name and account number to pu...","""[Provide name and account number]"""
3,Verify Identity,"""For security, can you confirm your billing ad...","""[Provide verification information]"""
4,Analyze Issue,"""Thank you for that information. Let me review...",-
5,Provide Solution,"""I've looked into your issue, and it appears t...","""Yes/No"""
6,Schedule Technician,"""Would you like me to schedule a technician to...","""[Provide date and time]"""
7,Confirm Resolution,"""I have scheduled the technician for [date/tim...","""No, thank you"" / ""Yes, [describe other issue]"""
8,Closing,"""Thank you for calling [Company Name]. Have a ...","""Thank you, goodbye"""


In [4]:
df_intention = pd.read_excel('../../data/intention_calls_ee.xlsx', sheet_name='intention1')
df_intention

,ID,Intention Call Categories,Definition or Description
0,1,Billing and Payments,"Questions regarding the latest bill, payment i..."
1,2,Service Requests and Appointments,"Scheduling maintenance or repair, connection r..."
2,3,Outages and Emergencies,"Reporting power outages, electrical emergencie..."
3,4,Account Management,"Updating personal information, closing/opening..."
4,5,Technical Support and Troubleshooting,"Help with understanding meter readings, assist..."
5,6,Complaints and Grievances,"Lodging a complaint regarding service, billing..."
6,7,Inquiries and General Information,"Information about services and products, guida..."
7,8,Renewable Energy and Sustainability Programs,"Information on renewable energy options, enrol..."
8,9,Contractor and Partner Communications,"Communication with third-party contractors, ve..."
9,10,Regulatory Compliance and Legal Inquiries,"Information regarding regulatory compliances, ..."


## Parameters for the company

## Template 1.1

In [5]:
prompt_call_template1 = ChatPromptTemplate.from_template(
    """
    The company is a electrical company that provides electrical services to residential and commercial customers.
    The name of the company is {company_name}.

    Generate a transcript of a call between two people for a call center for a electrical company.

    The conversation will be between a customer and a call center agent.

    Generate a random name and last name and for the customer and for the agent will be agent {agent_number}.

    The customer mood will be: {customer_mood}.

    The conversation should be between 5 and 10 minutes long with this intention or reason call: {intention}.

    The conversation is {type_resolution}.
    
    """
)

In [6]:
calls_list = []
calls_metadata = []
#generate a list of mood for the customer
mood_list = ["happy", "neutral", "angry","happy","neutral"] #Two times to increase the probability of happy and neutral

intention_list = df_intention['Intention Call Categories']
range_calls = 200
n_agent = 3
#generate a list of type of resolution
resolution_list = ["resolved", "resolved", "resolved", "unresolved"] #two time resolved to increase the probability of resolved

for i in range(range_calls):
    #generaintg random agent number from 1 
    agent_number = random.randint(1,n_agent)

    #generate a random mood from the mood list
    mood = random.choice(mood_list)
    intention = random.choice(intention_list)
    resolution = random.choice(resolution_list)

    call5 = prompt_call_template1.format_messages(company_name="ElectricVolt", 
                                                  customer_mood=mood, 
                                                  agent_number=agent_number,
                                                  intention=intention,
                                                  type_resolution=resolution)
    
    print("Generating Call Transcript #", i)
    response = chat_model(call5)
    calls_list.append(response.content)

    #append mood and agent number to the metadata
    calls_metadata.append([mood, agent_number, intention,resolution])

df_metadata = pd.DataFrame(calls_metadata)
df_calls = pd.DataFrame(calls_list)

df_calls = pd.concat([df_calls, df_metadata], axis=1)
df_calls.columns = ['call_transcript', 'mood', 'agent_number', 'intention', 'resolution']

now = datetime.datetime.now()

#change now to string with format YYYYMMDDhhmmss
now = now.strftime("%Y%m%d%H%M%S")
df_calls.to_csv('../../data/raw2/calls'+now+'.csv', index=False)

Generating Call Transcript # 0
Generating Call Transcript # 1
Generating Call Transcript # 2
Generating Call Transcript # 3
Generating Call Transcript # 4
Generating Call Transcript # 5
Generating Call Transcript # 6
Generating Call Transcript # 7
Generating Call Transcript # 8
Generating Call Transcript # 9
Generating Call Transcript # 10
Generating Call Transcript # 11
Generating Call Transcript # 12
Generating Call Transcript # 13
Generating Call Transcript # 14
Generating Call Transcript # 15
Generating Call Transcript # 16
Generating Call Transcript # 17
Generating Call Transcript # 18
Generating Call Transcript # 19
Generating Call Transcript # 20
Generating Call Transcript # 21
Generating Call Transcript # 22
Generating Call Transcript # 23
Generating Call Transcript # 24
Generating Call Transcript # 25
Generating Call Transcript # 26
Generating Call Transcript # 27
Generating Call Transcript # 28
Generating Call Transcript # 29
Generating Call Transcript # 30
Generating Call Tr

KeyboardInterrupt: 

In [ ]:
df_calls

,call_transcript,mood,agent_number,intention,resolution
0,[Phone rings]\n\nAgent 2: Thank you for callin...,angry,2,Complaints and Grievances,resolved
1,Agent 3: Thank you for calling ElectricVolt. M...,neutral,3,Accessibility and Special Needs,unresolved
2,"Agent 2: Thank you for calling ElectricVolt, m...",happy,2,Inquiries and General Information,resolved
3,"Agent 2: Thank you for calling ElectricVolt, m...",happy,2,Regulatory Compliance and Legal Inquiries,unresolved
4,"Agent 3: Thank you for calling ElectricVolt, m...",angry,3,Accessibility and Special Needs,resolved
...,...,...,...,...,...
95,"Agent 1: Thank you for calling ElectricVolt, m...",neutral,1,Technical Support and Troubleshooting,resolved
96,"Transcript:\n\nAgent 2: Good morning, thank yo...",angry,2,Contractor and Partner Communications,resolved
97,"Agent 1: Good morning, thank you for calling E...",angry,1,Contractor and Partner Communications,resolved
98,"Agent 1: Thank you for calling ElectricVolt, m...",neutral,1,Special Programs and Promotions,resolved
